In [ ]:
import os, sys, re, random, math, time, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans
import seaborn as sns
from pprint import pprint
import yaml
import uuid
import time

import anndata as ad
print(f"anndata=={ad.__version__}")
import scanpy as sc
import squidpy as sq
print(f"squidpy=={sq.__version__}")
import scimap as sm
print(f"scimap=={sm.__version__}")

from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import iqr

%matplotlib inline

In [ ]:
import sklearn
print(f"sklearn=={sklearn.__version__}")

In [ ]:
allClassData = pd.read_csv("./allClassData.csv")

In [ ]:
## Create AnnData object
noBad = allClassData[~allClassData['Class'].str.contains("ARTIFACT: ", na=False)] # remove artifacts from allClassData
singleJustVars = noBad.filter(regex='(_Cell_)',axis=1) # Get only markers
singleJustVars = singleJustVars[singleJustVars.columns.drop(list(singleJustVars.filter(regex='(_Max|_Mean)')))] # Remove max and mean

adata = ad.AnnData(singleJustVars) # create AnnData object
adata.obs_names = [ str(e) for e in noBad['uuid'].to_list()] # Set observation names
adata.var_names = singleJustVars.columns.to_list() # Set variable names
adata.obsm={ "spatial": noBad[['Centroid_X_um','invertY']].to_numpy(), # Add coordinates
             "Nucleus_Area" : noBad[['Nucleus_Area_um2']].to_numpy(), # Add nucleus area
             "Cell_Area" : noBad[['Cell_Area_um2']].to_numpy() # Add cell area
           }  
adata.obs["cell_type"] = pd.Categorical( noBad['Class'] ) # Add Class annotations
adata.uns["Slide"] = noBad["Slide"] # Add Slide
adata.obs["imageid"] = pd.Categorical( noBad["ROI"] ) # Add ROI number
adata.obs["cohort_site"] = pd.Categorical( noBad["Origin"] )

adata

In [ ]:
#  https://scimap.xyz/tutorials/5-Simple_Spatial_Analysis/

adata.obs["X"] = adata.obsm['spatial'][:,0] 
adata.obs["Y"] = adata.obsm['spatial'][:,1]

start_time = time.time()
# The function allows users to calculate the average shortest distance between phenotypes or clusters of interest (3D data supported).
adata = sm.tl.spatial_distance (adata, 
                               x_coordinate='X', y_coordinate='Y', 
                               z_coordinate=None, 
                               phenotype='cell_type', 
                               subset=None, 
                               imageid='imageid', 
                               label='spatial_distance')
print("--- %s minutes ---" % ((time.time() - start_time)/60))

In [ ]:
## Clean up Database entries
allClassData['cnt'] = 1
pivoted = allClassData.pivot(columns="Class", values="cnt")
pivoted.fillna(0, inplace=True)
cells = pd.concat([allClassData, pivoted], axis=1 )
#cells
print("cells Shape: {} x {}".format(cells.shape[0], cells.shape[1]))

In [ ]:
def get_windows(job,n_neighbors):
    '''
    For each region and each individual cell in dataset, return the indices of the nearest neighbors.
    'job:  meta data containing the start time,index of region, region name, indices of region in original dataframe
    n_neighbors:  the number of neighbors to find for each cell
    '''
    start_time,idx,tissue_name,indices = job
    job_start = time.time()
    print ("Starting:", str(idx+1)+'/'+str(len(exps)),': ' + exps[idx])

    tissue = tissue_group.get_group(tissue_name)
    to_fit = tissue.loc[indices][[X,Y]].values
    fit = NearestNeighbors(n_neighbors=n_neighbors).fit(tissue[[X,Y]].values)
    m = fit.kneighbors(to_fit)
    m = m[0], m[1]
    
    #sort_neighbors
    args = m[0].argsort(axis = 1)
    add = np.arange(m[1].shape[0])*m[1].shape[1]
    sorted_indices = m[1].flatten()[args+add[:,None]]

    neighbors = tissue.index.values[sorted_indices]
    end_time = time.time()
    print ("Finishing:", str(idx+1)+"/"+str(len(exps)),": "+ exps[idx],end_time-job_start,end_time-start_time)
    return neighbors.astype(np.int32)

In [ ]:
cluster_col = 'Class'
ks = [25,50] # k=5 means it collects 5 nearest neighbors for each center cell
X = 'Centroid_X_um'
Y = 'invertY'
reg = 'ROI' ## MUST BE CHARACTER COLUMN
keep_cols = [X,Y,reg,cluster_col]

#read in data and do some quick data rearrangement
n_neighbors = max(ks)
sum_cols = cells[cluster_col].unique()
values = cells[sum_cols].values ## This is giving 14, should be only 7 -- intentionally created by concat
# pprint([list(cells.columns),sum_cols,values, values.ndim, values.shape])

In [ ]:
#find windows for each cell in each tissue region
tissue_group = cells[[X,Y,reg]].groupby(reg)
exps = list(cells[reg].unique())
#pprint(tissue_group.groups.items())
tissue_chunks = [(time.time(),exps.index(t),t,a) for t,indices in tissue_group.groups.items() for a in np.array_split(indices,1)] 
tissues = [get_windows(job,n_neighbors) for job in tissue_chunks]

In [ ]:
#for each cell and its nearest neighbors, reshape and count the number of each cell type in those neighbors.
out_dict = {}
for k in ks:
    for neighbors,job in zip(tissues,tissue_chunks):
        chunk = np.arange(len(neighbors))#indices
        tissue_name = job[2]
        indices = job[3]
        window = values[neighbors[chunk,:k].flatten()].reshape(len(chunk),k,len(sum_cols)).sum(axis = 1)
        out_dict[(tissue_name,k)] = (window.astype(np.float16),indices)

#concatenate the summed windows and combine into one dataframe for each window size tested.
windows = {}
for k in ks:
    window = pd.concat([pd.DataFrame(out_dict[(exp,k)][0],index = out_dict[(exp,k)][1].astype(int),columns = sum_cols) for exp in exps],0)
    window = window.loc[cells.index.values]
    window = pd.concat([cells[keep_cols],window],1)
    windows[k] = window

In [ ]:
k = 50
n_neighborhoods = 9
neighborhood_name = "neighborhood"+str(k)
k_centroids = {}
print("k = "+str(k)+",  Communities = "+str(n_neighborhoods))

In [ ]:
windows2 = windows[k]
# windows2[cluster_col] = cells[cluster_col]
km = MiniBatchKMeans(n_clusters = n_neighborhoods,random_state=0,n_init=3)
labelskm = km.fit_predict(windows2[sum_cols].values)
k_centroids[k] = km.cluster_centers_
cells[neighborhood_name] = labelskm
cells[neighborhood_name] = cells[neighborhood_name].astype('category')

### Clusters - Proportions of Cell Types w/in a Cluster

In [ ]:
groupedvalues=cells.groupby(neighborhood_name).sum().reset_index()
gShort = groupedvalues[[neighborhood_name,'cnt']]
aggDF = cells.groupby([neighborhood_name,cluster_col]).size().reset_index()

merged_data= aggDF.merge(gShort, on=[neighborhood_name,neighborhood_name])
merged_data['percent'] = merged_data[0] / merged_data['cnt']
merged_data = merged_data.sort_values([neighborhood_name, "percent"], ascending = (True, False))

fltrDF = merged_data[merged_data['percent'] >= 0.1]
lastClst = '0'
namedClusters = {}
for index, row in fltrDF.iterrows():
    if lastClst != row[neighborhood_name]:
       # print("")
        namedClusters[row[neighborhood_name]] = [ "{:.0%} {}".format(row['percent'], row[cluster_col]) ]
    else:
        namedClusters[row[neighborhood_name]].append( "{:.0%} {}".format(row['percent'], row[cluster_col]) )
    print("Cluster {} contains {:.0%} {}".format(row[neighborhood_name], row['percent'], row[cluster_col]))
    lastClst = row[neighborhood_name]

for key, value in namedClusters.items():
    tmp = ', '.join(value).replace(' Cells','').replace(' Cell','')
    namedClusters[key] = "#"+str(key)+": "+tmp
    
#pprint(namedClusters)

In [ ]:
import matplotlib as mpl

merged_data # 
md2 = merged_data[['neighborhood50', 'Class', 0]]
md2 = np.log2(md2.pivot(index='neighborhood50', columns='Class')[0]+0.00001)
md2['ClusterNames'] = md2.index.map(namedClusters)
md2.set_index('ClusterNames', inplace=True)
sns.clustermap(md2, cmap = 'YlOrBr',row_cluster = True, col_cluster = False, figsize=(18, 8))

In [ ]:
customPalette = sns.set_palette(sns.color_palette( ["#e31a1c", "#fb9a99", "#b2df8a", "#d9c0b0",
                                                    "#1f78b4", "#cab2d6", "#33a02c", "#b15928",
                                                    "#fa7270"] ))

plt.figure(figsize=[7,6])
g=sns.barplot(y=neighborhood_name, x='cnt', data=groupedvalues, palette=customPalette)
for index, row in groupedvalues.iterrows():
    g.text( row.cnt, row.name, round(row.cnt), color='black', ha="left")

In [ ]:
cells[neighborhood_name] = cells[neighborhood_name].astype('category')

grps = cells["Cohort"].dropna().unique().tolist()
sts = cells["SiteLoc"].dropna().unique().tolist()
sts = [item for item in sts if item != 'NA']

pprint([grps,sts])

In [ ]:
gp1 = cells[(cells['Cohort']==grps[0]) & (cells['SiteLoc']==sts[0])]
lm = sns.lmplot(data = gp1, x = X, y=Y ,hue = neighborhood_name,palette = customPalette,
           height = 10,col = reg,col_wrap = 5,fit_reg = False)
fig = lm.fig 
# Add a title to the Figure
fig.suptitle("FOVs from {} - {}".format(grps[0],sts[0]), fontsize=12)

In [ ]:
gp1 = cells[(cells['Cohort']==grps[1]) & (cells['SiteLoc']==sts[0])]
lm = sns.lmplot(data = gp1, x = X, y=Y ,hue = neighborhood_name,palette = customPalette,
           height = 10,col = reg,col_wrap = 5,fit_reg = False)
fig = lm.fig 
# Add a title to the Figure
fig.suptitle("FOVs from {} - {}".format(grps[1],sts[0]), fontsize=12)

In [ ]:
gp1 = cells[(cells['Cohort']==grps[0]) & (cells['SiteLoc']==sts[1])]
lm = sns.lmplot(data = gp1, x = X, y=Y ,hue = neighborhood_name,palette = customPalette,
           height = 10,col = reg,col_wrap = 5,fit_reg = False)
fig = lm.fig 
# Add a title to the Figure
fig.suptitle("FOVs from {} - {}".format(grps[0],sts[1]), fontsize=12)

In [ ]:
gp1 = cells[(cells['Cohort']==grps[1]) & (cells['SiteLoc']==sts[1])]
lm = sns.lmplot(data = gp1, x = X, y=Y ,hue = neighborhood_name,palette = customPalette,
           height = 10,col = reg,col_wrap = 5,fit_reg = False)
fig = lm.fig 
# Add a title to the Figure
fig.suptitle("FOVs from {} - {}".format(grps[1],sts[1]), fontsize=12)

In [ ]:
cells['Cohort2'] = cells['Cohort'] + "-" + cells['SiteLoc']

# Set the context and style, and scale the font size
sns.set(context="notebook", font_scale=1.5, style = "white")
# Set the custom palette for the current plot
sns.set_palette(sns.color_palette(["#E86EA6", "#9E1A1A", "#39BADA", "#162ABF"]))

#plot for each group and each patient the percent of total cells allocated to each neighborhood
fc2 = cells.groupby(['Slide','Cohort2']).apply(lambda x: x[neighborhood_name].value_counts(sort = False,normalize = True))

fc2.columns = range(n_neighborhoods)
melt = pd.melt(fc2.reset_index(),id_vars = ['Slide','Cohort2'])
melt = melt.rename(columns = {'variable':'community','value':'frequency of communities'})
f,ax = plt.subplots(figsize = (16,9))
sns.stripplot(data = melt, hue = 'Cohort2',dodge = True,alpha = .4,x ='community', y ='frequency of communities')
sns.pointplot(data = melt, scatter_kws  = {'marker': 'd'},hue = 'Cohort2',dodge = .5,join = False,x ='community', y ='frequency of communities')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:4], labels[:4], title="Clinical Group",
          handletextpad=0, columnspacing=1,
          loc="upper right", ncol=2, frameon=True)

In [ ]:
cells.to_csv("./cells_9neighborhood.csv")